# Manipulate the Dispatch Model

This example shows how to manipulate the model,
such as trip a generator, change the load, etc.

In [1]:
import ams

import datetime

In [2]:
print("Last run time:", datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

print(f'ams:{ams.__version__}')

Last run time: 2023-11-28 22:06:37
ams:0.7.3.post74.dev0+g47c1ae3


In [3]:
ams.config_logger(stream_level=20)

## Run Simulations

### Load Case

In [4]:
sp = ams.load(ams.get_case('5bus/pjm5bus_uced.xlsx'),
              setup=True,
              no_output=True,)

Parsing input file "/Users/jinningwang/Documents/work/ams/ams/cases/5bus/pjm5bus_uced.xlsx"...
Input file parsed in 0.1106 seconds.
System set up in 0.0034 seconds.


The system load are defined in model `PQ`.

In [5]:
sp.PQ.as_df()

,idx,u,name,bus,Vn,p0,q0,vmax,vmin,owner
uid,,,,,,,,,,
0,PQ_1,1.0,PQ 1,1,230.0,3.0,0.9861,1.1,0.9,None
1,PQ_2,1.0,PQ 2,2,230.0,3.0,0.9861,1.1,0.9,None
2,PQ_3,1.0,PQ 3,3,230.0,4.0,1.3147,1.1,0.9,None


For simplicity, PQ load is typically marked as `pd` in RTED.

In [6]:
sp.RTED.pd.v

array([3., 3., 4.])

### Run Simulation

RTED can be solved and one can inspect the results as discussed in
previous example.

In [7]:
sp.RTED.run(solver='ECOS')

Routine <RTED> initialized in 0.0088 seconds.
RTED solved as optimal in 0.0170 seconds, converged after 9 iterations using solver ECOS.


True

Power generation (`pg`) and line flow (`plf`) can be accessed as follows.

In [8]:
sp.RTED.pg.v

array([2.1, 5.2, 0.7, 2. ])

In [9]:
sp.RTED.plf.v

array([ 0.70595331,  0.68616798,  0.00192539, -1.58809337,  0.61190663,
       -0.70192539,  0.70595331])

### Change Load

The load values can be manipulated in the model `PQ`.

In [10]:
sp.PQ.set(src='p0', attr='v', idx=['PQ_1', 'PQ_2'], value=[3.2, 3.2])

True

According parameters need to be updated to make the changes effective.
If not sure which parameters need to be updated, one can use
``update()`` to update all parameters.

In [11]:
sp.RTED.update('pd')

True

After manipulation, the routined can be solved again.

In [12]:
sp.RTED.run(solver='ECOS')

RTED solved as optimal in 0.0016 seconds, converged after 8 iterations using solver ECOS.


True

In [13]:
sp.RTED.pg.v

array([2.1       , 5.19999999, 1.10000002, 1.99999999])

### Trip a Generator

We can see that there are three PV generators in the system.

In [14]:
sp.PV.as_df()

,idx,u,name,Sn,Vn,bus,busr,p0,q0,pmax,...,Qc2min,Qc2max,Ragc,R10,R30,Rq,apf,pg0,td1,td2
uid,,,,,,,,,,,,,,,,,,,,,
0,PV_1,1.0,Alta,100.0,230.0,0,None,1.0000,0.0,2.1,...,0.0,0.0,999.0,999.0,999.0,999.0,0.0,0.0,0.5,0.0
1,PV_3,1.0,Solitude,100.0,230.0,2,None,3.2349,0.0,5.2,...,0.0,0.0,999.0,999.0,999.0,999.0,0.0,0.0,0.5,0.0
2,PV_5,1.0,Brighton,100.0,230.0,4,None,4.6651,0.0,6.0,...,0.0,0.0,999.0,999.0,999.0,999.0,0.0,0.0,0.5,0.0


`PV_1` is tripped by setting its connection status `u` to 0.

In [15]:
sp.StaticGen.set(src='u', attr='v', idx='PV_1', value=0)

True

In AMS, some parameters are defiend as constants in the numerical optimization model
to follow the CVXPY DCP and DPP rules.
Once non-parametric parameters are changed, the model needs to be resetup
to make the changes effective.

More details can be found at [CVXPY - Disciplined Convex Programming](https://www.cvxpy.org/tutorial/dcp/index.html#disciplined-convex-programming).

In [16]:
sp.RTED.update()

Resetup RTED OModel due to non-parametric change.


True

Then we can resolve the model.

In [17]:
sp.RTED.run(solver='ECOS')

RTED solved as optimal in 0.0171 seconds, converged after 8 iterations using solver ECOS.


True

In [18]:
sp.RTED.pg.v

array([0.        , 5.2       , 3.20000001, 2.        ])

In [19]:
sp.RTED.plf.v

array([ 0.79973461,  0.56088965, -2.16035886, -1.60053079,  0.39946921,
       -1.03964115,  0.79973461])

### Trip a Line

We can inspect the `Line` model to check the system topology.

In [20]:
sp.Line.as_df()

,idx,u,name,bus1,bus2,Sn,fn,Vn1,Vn2,r,...,tap,phi,rate_a,rate_b,rate_c,owner,xcoord,ycoord,amin,amax
uid,,,,,,,,,,,,,,,,,,,,,
0,0,1.0,Line 1-2,0,1,100.0,60.0,230.0,230.0,0.00281,...,1.0,0.0,999.0,999.0,999.0,None,None,None,-6.283185,6.283185
1,1,1.0,Line 1-4,0,3,100.0,60.0,230.0,230.0,0.00304,...,1.0,0.0,999.0,999.0,999.0,None,None,None,-6.283185,6.283185
2,2,1.0,Line 1-5,0,4,100.0,60.0,230.0,230.0,0.00064,...,1.0,0.0,999.0,999.0,999.0,None,None,None,-6.283185,6.283185
3,3,1.0,Line 2-3,1,2,100.0,60.0,230.0,230.0,0.00108,...,1.0,0.0,999.0,999.0,999.0,None,None,None,-6.283185,6.283185
4,4,1.0,Line 3-4,2,3,100.0,60.0,230.0,230.0,0.00297,...,1.0,0.0,999.0,999.0,999.0,None,None,None,-6.283185,6.283185
5,5,1.0,Line 4-5,3,4,100.0,60.0,230.0,230.0,0.00297,...,1.0,0.0,999.0,999.0,999.0,None,None,None,-6.283185,6.283185
6,6,1.0,Line 1-2 (2),0,1,100.0,60.0,230.0,230.0,0.00281,...,1.0,0.0,999.0,999.0,999.0,None,None,None,-6.283185,6.283185


Here line `1` is tripped by setting its connection status `u` to 0.

In [21]:
sp.Line.set(src='u', attr='v', idx=0, value=0)

True

In [22]:
sp.RTED.update()

Resetup RTED OModel due to non-parametric change.


True

In [23]:
sp.RTED.run(solver='ECOS')

RTED solved as optimal in 0.0190 seconds, converged after 8 iterations using solver ECOS.


True

Here we can see the tripped line has no flow.

In [24]:
sp.RTED.plf.v

array([-0.        ,  0.70423831, -2.03964421, -1.8645941 ,  0.13540589,
       -1.16035581,  1.3354059 ])